In [17]:
import os 

os.chdir(r'C:\Users\pc\Downloads\FaceDetection')


In [3]:
import cv2
from simple import SimpleFacerec

In [4]:
import face_recognition


In [13]:
import face_recognition
import cv2
import os
import glob
import numpy as np

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []

        # Resize frame for faster processing
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        """
        Load images from the provided directory and encode faces.
        :param images_path: Directory path to images
        """
        images_path = glob.glob(os.path.join(images_path, "*.*"))
        print(f"{len(images_path)} encoding images found.")

        # Process and encode each image
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Extract the filename (without extension) for the name
            basename = os.path.basename(img_path)
            filename, _ = os.path.splitext(basename)

            # Get the face encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store the encoding and corresponding name
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded successfully.")

    def detect_known_faces(self, frame):
        """
        Detect known faces in a given frame.
        :param frame: Video frame to process
        :return: face locations and their corresponding names
        """
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the resized frame
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # Compare detected face with known faces
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # Proceed only if known encodings exist
            if len(self.known_face_encodings) > 0:
                # Get the face distance
                face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)

                # Ensure face_distances is not empty
                if len(face_distances) > 0:
                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = self.known_face_names[best_match_index]

            face_names.append(name)

        # Scale face locations back to original frame size
        face_locations = np.array(face_locations) / self.frame_resizing
        return face_locations.astype(int), face_names

    def process_video_stream(self, video_source):
        """
        Process the video stream to detect known faces in real-time.
        :param video_source: Video source (camera index or video file)
        """
        cap = cv2.VideoCapture(video_source)

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            # Detect known faces in the frame
            face_locations, face_names = self.detect_known_faces(frame)

            # Draw boxes around faces and label them
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                # Draw a rectangle around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

                # Label the face
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)

            # Display the result
            cv2.imshow("Video", frame)

            # Break on 'q' key press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release the video capture and close windows
        cap.release()
        cv2.destroyAllWindows()


In [16]:
sfr = SimpleFacerec()
sfr.load_encoding_images("images")
sfr.process_video_stream(0)  

3 encoding images found.
Encoding images loaded successfully.
